# Serializing and Deserializing datetimes

## Serializing datetimes with time zones

In the previous two workbooks, we have focused on *timestamps*, which can be safely stored as some fixed point relative to UTC, since they are fixed point in time. When working with *civil times*, however, the datetime does have a fixed mapping to absolute time, but rather has a relationship that may change over time if the rules for time zone offsets change.

As such, you do **not** want to serialize your datetime as an epoch time, a time in UTC or a datetime with a fixed offset from UTC, but rather you want to store the "wall time" and the time zone (or more likely, some key into a lookup table of time zone rules).

In [1]:
import sd_answers
from sd_helpers import ChileTzInfo
from datetime import datetime, timezone

In [2]:
dt = datetime(2016, 6, 1, 13, 30, tzinfo=ChileTzInfo(access_date=datetime(2016, 3, 10)))
print(dt)

2016-06-01 13:30:00-03:00


In [3]:
dt_utc = dt.astimezone(timezone.utc)
print(dt_utc)

2016-06-01 16:30:00+00:00


In [4]:
dt_after = dt_utc.astimezone(ChileTzInfo(access_date=datetime(2016, 5, 24)))
print(dt_after)

2016-06-01 12:30:00-04:00


If instead, we were to serialize this with a function that stored the thing that we *actually* care about (the wall time), and store the time zone, we would not have had this problem:

In [5]:
dt_local = dt.replace(tzinfo=None)
print(dt_local)

2016-06-01 13:30:00


In [6]:
dt_after = dt_local.replace(tzinfo=ChileTzInfo(access_date=datetime(2016, 5, 24)))
print(dt_after)

2016-06-01 13:30:00-04:00


### Exercise: Write a JSON encoder and decoder hook for datetimes


In [7]:
import json

from dateutil import tz

def get_annotated_tz(name):
    """
    There is currently no supported way to get a string that can
    be passed to `gettz` from the `tzinfo` object itself, so until
    that is supported, hack this feature in.
    """
    tzi = tz.gettz(name)
    if tzi is not tz.UTC:
        tzi.name = name

    return tzi

class DatetimeEncoder(json.JSONEncoder):
    def default(self, obj):
        return sd_answers.DatetimeEncoder().default(obj)

In [8]:
# Tests
dt1 = datetime(2020, 1, 1, tzinfo=get_annotated_tz('America/New_York'))
dt2 = datetime(2020, 1, 1)
dt3 = datetime(2020, 1, 1, 14, 31, 11, 123456, tzinfo=get_annotated_tz('UTC'))
dt4 = datetime(2020, 1, 1, 14, tzinfo=timezone.utc)
dt5 = datetime(2020, 11, 1, 1, 30, fold=1, tzinfo=get_annotated_tz('America/New_York'))

encoder = DatetimeEncoder()

print(encoder.encode(dt1))
print(encoder.encode(dt2))
print(encoder.encode(dt3))
print(encoder.encode(dt4))
print(encoder.encode(dt5))

{"datetime": "2020-01-01T00:00:00", "fold": 0, "timezone": "America/New_York"}
{"datetime": "2020-01-01T00:00:00", "fold": 0, "timezone": null}
{"datetime": "2020-01-01T14:31:11.123456", "fold": 0, "timezone": "UTC"}
{"datetime": "2020-01-01T14:00:00", "fold": 0, "timezone": "UTC"}
{"datetime": "2020-11-01T01:30:00", "fold": 1, "timezone": "America/New_York"}


In [9]:
def decode_datetime_hook(obj):
    return sd_answers.decode_datetime_hook(obj)


In [10]:
# Tests
decoder = json.JSONDecoder(object_hook=decode_datetime_hook)

for dt in (dt1, dt2, dt3, dt4, dt5):
    dt_rt = decoder.decode(encoder.encode(dt))
    assert dt_rt == dt
    
print("Passed")

Passed
